In [1]:
import pandas as pd
import requests
import formula1 as f1

In [2]:
# Accessing the Ergast F1 API. This was especially complex and could only be done in python due to the multiple levels 
# of nesting (MRData, StandingsTable, StandingsLists etc) present in the API
# Inspired by the pyErgast python module

def constructor_standings(year):
    url = 'http://ergast.com/api/f1/{}/constructorStandings.json?limit=1000000'.format(year)
    r = requests.get(url)
    constructorStandings = r.json()['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']
    for i in constructorStandings:
        i['constructorID'] = i['Constructor']['constructorId']
        i['name'] = i['Constructor']['name']
        del i['Constructor']
    return pd.DataFrame(constructorStandings)

In [3]:
# Create a dataframe with the year and number of races in that year to normalise the total points scored.
#race_count = pd.DataFrame(columns={'year', 'races'})
race_count = {}
for i in range(2012, 2021):
    df = f1.get_schedule(i)
    #race_count.loc[(i-2012)] = [i, len(df['round'])]
    race_count[i] = len(df['round'])*(25+18+1)

# The maximum points that a constructor can score in a single race is 25 (first place) + 18 (second place) + 1 (fastest lap) (since each team has 2 drivers)
#race_count['maxpoints'] = race_count['races'].values*(25+18+1)

In [4]:
master_data = pd.DataFrame()

for i in range(2012, 2022):
    df = constructor_standings(i)
    df = df[['name', 'points', 'constructorID']]
    df['year'] = i
    master_data = master_data.append(df)


In [ ]:
constructor_list = ['mercedes', 'red_bull', 'ferrari', 'williams', 'mclaren', 'alpine', 'alphatauri', 'alfa', 'aston_martin']


In [ ]:
master_data['constructorID'].replace("lotus_f1", "alpine", inplace=True)
master_data['constructorID'].replace("renault", "alpine", inplace=True)

master_data['constructorID'].replace("sauber", "alfa", inplace=True)

master_data['constructorID'].replace("toro_rosso", "alphatauri", inplace=True)

master_data['constructorID'].replace("force_india", "aston_martin", inplace=True)
master_data['constructorID'].replace("racing_point", "aston_martin", inplace=True)

In [ ]:
master_data['name'].replace("Lotus F1", "Alpine F1 Team", inplace=True)
master_data['name'].replace("Renault", "Alpine F1 Team", inplace=True)

master_data['name'].replace("Sauber", "Alfa Romeo", inplace=True)

master_data['name'].replace("Toro Rosso", "AlphaTauri", inplace=True)

master_data['name'].replace("Force India", "Aston Martin", inplace=True)
master_data['name'].replace("Racing Point", "Aston Martin", inplace=True)

In [ ]:
master_data = master_data[master_data['constructorID'].isin(constructor_list)]

In [ ]:
master_data.reset_index(drop=True)
master_data = master_data[master_data['year'] != 2021]

In [ ]:
def categorise(row):
    return race_count[row['year']]

master_data['maxpoints'] = master_data.apply(lambda row: categorise(row), axis=1)

In [ ]:
master_data

In [ ]:
#master_data.to_csv('chart2-constructor-standings-2012-2021.csv')
#master_data.to_excel('chart2-constructor-standings-2012-2021.xlsx')